# Learning Robotics on the Jetson TX2

In this session, you will be using the Jetson TX2 to control a robot arm. This robot arm will interact with a panel of buttons and LEDs. 
This session will be broken into two main parts: ***prepare and test your hardware***: take control of the arm, leds and buttons with the Jetson, and then ***add reinforcement learning*** to teach this robot to play a game. 

In this first part of the session, you will learn about different hardware components, including LEDs, buttons, and servo motors, how to assemble them for this project, and two different ways that hardware can interface with the Jetson.

This part of the notebook contains some extra background information on each component and communication method that is not required to complete this project, but it is here for your reference and interest. 


- Step 1: [Required components](#breadboard)
- Step 2: [Wiring diagram and assembly instructions](#instructions)

- Step 2 : Communication with the Jetson
 - [GPIO](#gpio)
 - [I2C](#i2c)
- Step 3: [Testing](#testing)




<a id='breadboard'></a>
## Breadboard 
For this project you’ll need a full size solderless breadboard so that you can reuse the components. In a breadboard like the one shown below, the + and - “rails” are internally connected. The interior numbered groups are also internally connected.

![Breadboard](images/image9.png)

## Jumper wires 
To make connections between the Jetson and other components, we will require wires. You will have two types: Male/Female; One end connects to select GPIO pins and the other goes into the breadboard, and Male/Male; to connect breadboard to breadboard.

![Jumper wires](images/image2.jpg)

## Light-Emitting Diodes (LEDs)
For this project, we will be using four 10mm LEDs. LEDs have an anode (the long lead of the LED) and cathode(the short lead of the LED), and the wires connected to them for this project are colour coded (red is positive and black is negative). ***It is important to plug in an LED in the correct orientation, as diodes only work in one direction. So keep an eye on this colour coding in the wiring diagram in the steps below.***

LEDs can blow if the circuit has not been connected properly and too much current is put through the LED, which is why we will use resistors in the LED circuit.

![LEDs](images/image7.jpg)


**Additional resources:**
- [More about LEDs](https://learn.sparkfun.com/tutorials/light-emitting-diodes-leds)


## Resistors 

Resistors are needed in a circuit to divide the voltage of a circuit and to control the current flow. We'll need resistors for the LED circuits, the transistor switch, and another for the button circuit. The value of resistor or resistors that you need varies depending on your other specific circuit components. You can identify the value of the resistor based on the colour coded stripes, which you will be able to see in the wiring diagram provided.

![Resistors](images/image8.png)

**Additional Resources:**
- [More about resistors from Sparkfun](https://learn.sparkfun.com/tutorials/resistors/take-a-stance-the-resist-stance)

## Transistor
For this project, we’ll use transistors as electronic switches. That means we’ll hook it up to a line we can control with the software and it will in turn make the circuit turn “on” and “off” by closing and opening the circuit, just like a physical switch. The transistor circuit allows us to control the LED with a very small output signal from the system, even one that is too small to drive an LED directly. We will be using small signal NPN transistor [2N2222A](https://en.wikipedia.org/wiki/2N2222) or similar.

![Transistor](images/image5.png)

**Additional Resources:**
- [Sparkfun: NPN (BC337)](https://www.sparkfun.com/products/13689)

## Button

![Button](images/image6.png)

A button or switch is used to control the current flow in a circuit by being in an open or closed state. When the switch is open or off, the circuit is open and current is not able to flow. When the switch is closed or on, the circuit is closed and current flows through.

These physical switches are momentary, which means when they are active only when they are pressed. This is in contrast with a maintained switch, where if you toggle the switch on, it will remain on until you press it a second time.


## Pull-Down Resistors

The button circuits we are building allow for the Jetson to detect the state of the button input. We also will require something called a pull-down resistor, where when the switch is in an open state, the circuit will be “pulled down” to ground. This prevents something called a “floating pin”, where the signal of the Jetson pin floats between 0 and 1 when the button is not pressed and the circuit is open. With a pull-down resistor, the pin will consistently read 0 when the button is open, and 1 when the button is closed.

[This](https://learn.sparkfun.com/tutorials/pull-up-resistors) is a great article about pull-up resistors, where the principle is the same, except in this project we are using a resistor that “pulls down” to ground.

## Servo motors

Servos are motors that can be programmed to move to specific positions, controlled by sending electrical pulses with variable width, or pulse width modulation (PWM). The length of the pulse determines how far the motor will turn.
 
Most hobby servos use a standard type of 3-pin plug, with the same control signaling.
The connector is a female, 3-pin header. The wiring is color coded, but the colors aren’t always the same. The red wire is positive, the brown or black wire is negative, and the white orange or yellow wire is the PWM control signal.

![Servo motor](images/image4.png)

**Additional Resources:**

https://learn.sparkfun.com/tutorials/hobby-servo-tutorial

https://learn.sparkfun.com/tutorials/pulse-width-modulation

https://www.jameco.com/jameco/workshop/howitworks/how-servo-motors-work.html

## PWM Driver -  Servo HAT

Unfortunately we can’t use the PWM GPIO pins on the Jetson to control the servos. This is because PWM requires precise clock and refresh, which is’t something that the Jetson can do very well - considering it is also running a non real-time operating system.
Using I2C (only two command pins) , which we will discuss further later, the breakout board can control 16 free-running PWM outputs. The SparkFun Pi Servo HAT allows control of up to 16 servo motors using an I2C connection.

We will learn about the code to communicate with this board using I2C later in this project.

![PWM Driver - Servo Hat](images/image10.png)

<a id='instructions'></a>

## Wiring Diagram and Assembly Instructions

**Step 1**: Read all of the assembly instructions and look at all images before starting! Please ask an instructor or TA for assistance if anything is unclear. 

**Step 2**: Wire the LEDs and buttons to the Jetson as shown in the wiring diagram below. Instead of wiring directly to the Jetson's J21 header, we will be wiring to the Sparkfun Servo HAT header **not** plugged in to the board. This is for the board's safety, and will give you a chance to check your wiring before proceeding with plugging the Servo HAT into the Jetson. Please  see the photos below for proper orientation of the Servo HAT relative to the Jetson headers and the wiring diagram.
![Jetson](images/jetson3.jpg)

![Wiring diagram](images/image3.jpg)

![Jetson](images/jetson5.jpg)

**Step 3**: CHECK AND DOUBLE CHECK YOUR WIRING. Especially any wires that are coloured red or black in the diagram.

**Step 4** Stack the red PWM Servo board on to the J21 Header of the TX2.
![Jetson](images/jetson1.jpg)

![Jetson](images/jetson2.jpg)

<a id='gpio'></a>

## Communication With the Jetson: GPIO

General Purpose Input/Output (GPIO) is a basic form of communicating with pins to turn them on or off, or read the state of the pin. When configured as an input, the pin can be queried and it will return a 1 or 0. When configured as an output, the pin can be set to either 1 or 0.

## Control GPIO pins with software

The low level hardware interface for the GPIO pins is controlled in user space through a pseudo file system in Linux called sysfs. Control of the pin must first be "exported" to user space from kernel space. After that, the direction and pin value can be accessed like any other file. On the Jetson TX2, pin 18 corresponds to gpio481, so the number that must be exported to control it is 481.

![Control GPIO Pins](images/image1.png)

Begin by moving to sudo control and exporting the GPIO sysfs number. To control pin 18 of J21 on the Jetson TX2 Development board, the number is 481.

```
$ sudo -s

# cd /sys/class/gpio

# echo 481 >export
```

If you list the contents of the directory at this point, there is now a new directory named `gpio481`, and under it are a number of virtual files to control pin 18, including `direction`, to control whether the pin is used as an input or output (in or out) and a `value` file to control or read the voltage as high or low (1 or 0). To toggle the pin, declare it as an output and then output the value alternately as high or low:

```
# echo out >gpio481/direction

# echo 1 >gpio481/value

# echo 0 >gpio481/value

# echo 1 >gpio481/value

# echo 0 >gpio481/value
```

## GPIO pins

The same procedure can be used to control other GPIO pins. The following is a table of sysfs and pin mappings.

Sysfs GPIO | Name | Header | Pin
------------ | ------------- | ------------ | ------------- 
gpio481 | GPIO16_MDM_WAKE_AP Modem Wake AP GPIO | J21 | 18
gpio398 | GPIO19_AUD_RST Audio Reset (1.8/3.3V) | J21 | 29
gpio298 | GPIO9_MOTION_INT MotionINterrupt (3.3V) | J21 | 31
gpio389 | GPIO11_AP_WAKE_BT AP Wake Bt GPIO | J21 | 33
gpio388 | GPIO9_ALS_PROX_INT(3.3V) | J21 | 37
gpio488 | GPIO15_AP2MDM_READY AP to Modem Ready GPIO | J26 | 6
gpio299 | CAN_WAKE | J26 | 19
gpio300 | CAN1_ERR | J26 | 1

## GPIO library

Now that you understand how GPIO pins are controlled, we will be using a Python library called gpio for our code. This library provides GPIO access via the standard linux [sysfs interface](https://www.kernel.org/doc/Documentation/gpio/sysfs.txt)

**Additional Resources:**

https://pypi.python.org/pypi/gpio/0.2.0

<a id='i2c'></a>

## Communication With the Jetson: I2C

I2C is a more complex communication protocol, where 2 pins are used, but multiple devices can be controlled on a single bus, and two way communication is possible.

A data line and clock signal - SCL keeps two devices synchronized during data transfer
SDA is used to exchange data between two devices, and the master device ensures that only one device is using the communication line at one time.

I2C protocol uses device addresses to coordinate devices and traffic on the data line.

In most OS level interactions, the I2C bus is referred to as SMBus. We will use the Python library smbus to create SMBus object and attach it to a bus on the Jetson. We will need to tell the program the servo hat’s I2C address, which is 0x40 by default. We will enable the PWM chip, and write values to the PWM registers to assign a start time and stop time to the servo hat channels. Each channel on the servo hat controls one single servo, and each of these channels has a unique start and stop address to write values to. The start time and stop time determine when the pulse of the PWM goes high and low. The control signal of the servo is taken from this pulse width.

For this project, we will be using two servo motors. See the table below for the start and stop addresses for channel 0 and 1 of the servo hat, which you will see used in the code.

Channel # | Start Address | Stop Address
------------ | ------------- | ------------
Ch 0 | 0x06 | 0x08
Ch 1 | 0x0A | 0x0C

From the Servo HAT documentation: "If you write a 0 to the start address, every degree of offset from 90 degrees requires 4.6 counts written to the stop address. In other words, multiply the number of degrees offset from neutral you wish to achieve by 4.6, then either add or subtract that result from 1250, depending on the direction of motion you wish. For example, a 45 degree offset from center would be 207 (45x4.6) counts either more or less than 1250, depending upon the direction you desire the motion to be in."

**Additional Resources:**

https://learn.sparkfun.com/tutorials/i2c

<a id='testing'></a>
## Testing Your Wiring and Calibration of Your Robot Arm

Download gtc_arm.py on your Jetson, and run using:

```
sudo python gtc_arm.py
```

Each light should turn on, one at a time from left to right. Once the first light turns on, the arm will turn to that position, and it will attempt to press the button. If the button is pressed, the light should turn off. This will repeat for each light/ button combo. 

If one (or more) of the lights does not turn on - check your wiring.

If the button is successfully pressed by the arm but the light does not turn off - check your wiring. 

If the arm misses the button - calibrate the position the base servo turns to. 

TODO: link to next notebook